## Description 
Basically u can visualise the regions we are trying to study. 

Select the year to get yearwise grids, change the gridsize incase the datset resolution changes. 

- Red = Treated (Disaster occured)
- Green = Untreated (No disaster) 


In [ ]:
%pip install -q pandas folium ipywidgets ipython

In [ ]:
import pandas as pd
import os
from re import T
base_path= 'data/processed/'
file_path= os.path.join(base_path, "analysis_panel.csv")
df = pd.read_csv(file_path)
df.drop(columns=['first_treatment_year'], inplace = True)
df['treated'] = df['treated'].astype(int)



In [ ]:

# Group by year and count treated/untreated cases
treated_counts_per_year = df.groupby('year')['treated'].value_counts().unstack(fill_value=0)

display(treated_counts_per_year)

# Folium Overlay to visualize
Create a Folium map overlay for the locations in the dataframe, allowing year-wise selection to highlight grids and display the species they contain.

## Folium Visualizer

Change **grid_size** to whatever grid resolution we use, this is in km btw. 

Eg for 11 x 11 grids: **grid_size = 11** 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

import folium

# Approximate coordinates for centering on Asia
asia_latitude = 23.4
asia_longitude = 84.2
initial_zoom = 3

grid_size = 11  # degrees - this is in degrees adjust as needed 

grid_size = grid_size / 111  # convert to degrees (approx) - dont change this line
# Create a Folium map object
m = folium.Map(location=[asia_latitude, asia_longitude], zoom_start=initial_zoom)

# # Display the map (optional, but good for verification)
# m

unique_years = sorted(df['year'].unique())
year_dropdown = widgets.Dropdown(
    options=unique_years,
    description='Select Year:',
    disabled=False,
)

output = widgets.Output()
display(year_dropdown, output)

def filter_by_year(df, year):
  filtered_df = df[df['year'] == year].copy()
  return filtered_df

def update_map(change):
    with output:
        clear_output(wait=True)
        selected_year = change['new']
        filtered_df = filter_by_year(df, selected_year)

        m = folium.Map(location=[asia_latitude, asia_longitude], zoom_start=initial_zoom)

        for _, row in filtered_df.iterrows():
            lat, lon = row['lat_center'], row['lon_center']
            treated, species = row['treated'], row['species']


            sw = [lat - grid_size/2, lon - grid_size/2]
            ne = [lat + grid_size/2, lon + grid_size/2]     

            color = 'red' if treated == 1 else 'green'

            folium.Rectangle(
                bounds=[sw, ne],
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.5,
                popup=f"Species: {species}<br>Treated: {'Yes' if treated==1 else 'No'}"
            ).add_to(m)

        m._repr_html_()  # ✅ forces rendering
        display(m)

year_dropdown.observe(update_map, names='value')

# ✅ trigger once on load
update_map({'new': year_dropdown.value})
